# Versão 1.1

Nessa versão utilizei K-means para os 6 clusteres, dados normalizados somente por feature e NÃO utilizei o filtro Kalman.

Coluna 'cycle_num' dos dados forma normalizados ( value / dataset.['cycle_num'].max() ) e depois multiplicando pelo valor novamente ao avaliar na métrica da competição. (melhora de 8%)

Outra modificação foi a inclusão desta coluna no treinamento do modelo. O resultado foi absurdamente melhorado. (imensurável a melhora no modelo)

In [1]:
#Importando as bibliotecas

# Data Analysis Tools
import pandas as pd
import numpy as np
from time import time
import logging as log

# Visualization Tools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

# Machine Learning Tools
import sklearn
import sklearn.mixture as mix
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn import preprocessing
import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
import pickle


#Scoreing tools
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
# from sklearn.metrics import mean_squared_log_error

# plt.rcParams['figure.figsize'] = (10, 10)

def get_sensor_names(tag_names):
    """ Get tagnames starting with sensor.
    
    :param tag_names: Input time series data frame
    
    :return list of string tag names starting with sensor.
    """
    
    return [tag_name for tag_name in tag_names if tag_name.startswith('sensor')]

c:\programdata\anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\programdata\anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [14]:
#Import dos dados

#Dataset de treino
train_final_data = pd.read_csv('train_final_data.csv', sep=',')
target_final_data_train_scaled = pd.read_csv('target_final_data_train_scaled.csv', sep=',', names = ['cycle_num'],header = None)

#Dataset de teste
test_final_data = pd.read_csv('test_final_data.csv', sep=',')
target_final_data_test_scaled = pd.read_csv('target_final_data_test_scaled.csv', sep=',', names = ['cycle_num'],header = None)

#Dataset de teste final
finaltest_final_data = pd.read_csv('finaltest_final_data.csv', sep=',')
target_final_data_finaltest_scaled = pd.read_csv('target_final_data_finaltest_scaled.csv', sep=',', names = ['cycle_num'],header = None)

print "Os dados dos sensores e settings:"
print "O dataset de treino tem {} linhas de dados e {} atributos.".format(*train_final_data.shape)
print "O dataset de teste tem {} linhas de dados e {} atributos.".format(*test_final_data.shape)
print "O dataset de testes finais tem {} linhas de dados e {} atributos.".format(*finaltest_final_data.shape)

print "\n"

print "Os dados dos sensores e settings:"
print "O dataset de treino tem {} linhas de dados e {} atributos.".format(*target_final_data_train_scaled.shape)
print "O dataset de teste tem {} linhas de dados e {} atributos.".format(*target_final_data_test_scaled.shape)
print "O dataset de testes finais tem {} linhas de dados e {} atributos.".format(*target_final_data_finaltest_scaled.shape)


Os dados dos sensores e settings:
O dataset de treino tem 45918 linhas de dados e 20 atributos.
O dataset de teste tem 29820 linhas de dados e 20 atributos.
O dataset de testes finais tem 55156 linhas de dados e 20 atributos.


Os dados dos sensores e settings:
O dataset de treino tem 45918 linhas de dados e 1 atributos.
O dataset de teste tem 29820 linhas de dados e 1 atributos.
O dataset de testes finais tem 55156 linhas de dados e 1 atributos.


In [24]:
train_final_data = train_final_data.drop(train_final_data.columns[[0]], axis = 1)
test_final_data = train_final_data.drop(train_final_data.columns[[0]], axis = 1)
finaltest_final_data = finaltest_final_data.drop(finaltest_final_data.columns[[0]] ,axis = 1)

In [27]:
finaltest_final_data.head()

,overall setting,sensor2,sensor3,sensor4,sensor6,sensor7,sensor8,sensor9,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor20,"sensor9,14_break"
0,0,0.530201,0.351019,0.551776,0.0,0.396739,0.138889,0.151036,0.0,0.291339,0.616949,0.148148,0.186259,0.544818,0.0,0.444444,0.369565,0
1,0,0.459732,0.164401,0.457852,1.0,0.519022,0.111111,0.134116,0.0,0.354331,0.467797,0.157407,0.131014,0.571429,0.0,0.444444,0.586957,0
2,0,0.419463,0.352588,0.389602,1.0,0.703804,0.111111,0.132873,1.0,0.377953,0.352542,0.129630,0.211785,0.371148,0.0,0.333333,0.489130,0
3,0,0.587248,0.484579,0.235344,0.0,0.663043,0.129630,0.149448,1.0,0.370079,0.467797,0.129630,0.174342,0.299253,0.0,0.444444,0.413043,0
4,0,0.506711,0.525614,0.487805,0.0,0.551630,0.148148,0.185221,1.0,0.275591,0.406780,0.138889,0.159335,0.571895,0.0,0.444444,0.695652,0


# Modelos de regressão

In [ ]:
# #ATENÇÃO------------->>>>>>>>>>> "Teste com o cross_val_predict para verificar os dados? Entender o que essa função realiza"
# from xgboost import XGBRegressor as xgb
# lr = linear_model.LinearRegression()
# # lr = xgb(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7)
# boston = train_final_data
# y = target_final_data

# # cross_val_predict returns an array of the same size as `y` where each entry
# # is a prediction obtained by cross validation:
# predicted = cross_val_predict(lr, train_final_data, target_final_data, cv=100)

# fig, ax = plt.subplots(figsize=(10,10))
# plt.title(lr.__class__.__name__,fontsize=30)
# ax.scatter(y, predicted, c=('b', 'k'))
# ax.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=4)
# ax.set_xlabel('Measured',fontsize=25)
# ax.set_ylabel('Predicted',fontsize=25)
# ax.tick_params(labelsize=20)
# plt.savefig(lr.__class__.__name__+'_Modelo.png', format='png', dpi=100)
# plt.show()

In [28]:
#Funções para regressão dos dados:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
     
    clf.fit(X_train, y_train)
    print "O modelo foi treinado!"
    return 

def predict_labels_train(clf, X, y):
    ''' Faz uma estimativa utilizando um modelo de regressão ajustado em algum score ainda a ser definido. '''
       
    y_pred = clf.predict(X)
    
    
    # Imprime os resultados de retorno
    print ("Pronto!\n As previsões de treino para o modelo {} foram feitas!".format(clf.__class__.__name__))
    return  {"R2 score": r2_score(y.values, y_pred), 
             "Explained Variance Score": explained_variance_score(y.values, y_pred),
             "Mean Squared Error": mean_squared_error(y.values, y_pred)
            }

def predict_labels_test(clf, X, y):
    ''' Faz uma estimativa utilizando um modelo de regressão ajustado em algum score ainda a ser definido. '''
       
    y_pred = clf.predict(X)

    
    
    
    # Imprime os resultados de retorno
    print ("Pronto!\n As previsões de teste para o modelo {} foram feitas!\n".format(clf.__class__.__name__))
    return  {"R2 score": r2_score(y.values, y_pred), 
             "Explained Variance Score": explained_variance_score(y.values, y_pred),
             "Mean Squared Error": mean_squared_error(y.values, y_pred)
                          }

# def metrica(d,a1,a2)

#     for i, x in enumerate(d):
#         if (x < 0):
#             np.exp(-(d/a1))
#         else:
#             np.exp(d/a2)

In [30]:
# Neste passo irei realizar os modelos de regressão e mostrar os resultados

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import linear_model
from xgboost import XGBRegressor as xgb

Regression = [DecisionTreeRegressor(),SVR(C=1.0, epsilon=0.2) ,KNeighborsRegressor(n_neighbors=2,algorithm = 'auto'),linear_model.LinearRegression(),
              xgb(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7) 
             ]

resultados = { 
        'Modelo utilizado': [],
        'Conjunto de Treino': [], 
        'Conjunto de Teste': [],
        'Pontuação R2 (treinamento)': [], 
        'Pontuação R2 (teste)': [],
        'Explained Variance Score (treinamento)':[],
        'Explained Variance Score (teste)':[],
        'Mean Squared Error (treinamento)':[],
        'Mean Squared Error (teste)':[]
            }

################################DADOS PASSADOS NO MODELO###########################################################
# ##Treino
# #Coluna targe do dataset
# target_final_data_train = train_final_data['cycle_num']
# #Colunas com os dados
# train_final_data = train_final_data.drop(['cycle_num'], axis = 1)

##################################################################################################################

Dados_normalizados = [train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)]

for clf in Regression:
    for split in Dados_normalizados:
        
        print ("Próximo modelo...\n")
        X_train, X_test, y_train, y_test = split
        
        time_train = train_classifier(clf, X_train, y_train)
        
        time_predict_train = predict_labels_train(clf, X_train, y_train)
        
        time_predict_test = predict_labels_test(clf, X_test, y_test)
        
        resultados['Modelo utilizado'].append(clf.__class__.__name__)
        resultados['Conjunto de Treino'].append(X_train.shape)
        resultados['Conjunto de Teste'].append(X_test.shape)
        resultados['Pontuação R2 (treinamento)'].append(time_predict_train["R2 score"])
        resultados['Pontuação R2 (teste)'].append(time_predict_test["R2 score"])
        resultados['Explained Variance Score (treinamento)'].append(time_predict_train["Explained Variance Score"])
        resultados['Explained Variance Score (teste)'].append(time_predict_test["Explained Variance Score"])
        resultados['Mean Squared Error (treinamento)'].append(time_predict_train["Mean Squared Error"])
        resultados['Mean Squared Error (teste)'].append(time_predict_test["Mean Squared Error"])
                
                
Results = pd.DataFrame(resultados)
Results = Results[['Modelo utilizado','Pontuação R2 (treinamento)','Explained Variance Score (treinamento)','Mean Squared Error (treinamento)',
                  'Pontuação R2 (teste)','Explained Variance Score (teste)','Mean Squared Error (teste)','Conjunto de Treino','Conjunto de Teste']]
Results.head()


Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo DecisionTreeRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo DecisionTreeRegressor foram feitas!

Próximo modelo...



c:\programdata\anaconda2\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo SVR foram feitas!
Pronto!
 As previsões de teste para o modelo SVR foram feitas!

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo KNeighborsRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo KNeighborsRegressor foram feitas!

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo LinearRegression foram feitas!
Pronto!
 As previsões de teste para o modelo LinearRegression foram feitas!

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo XGBRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo XGBRegressor foram feitas!



,Modelo utilizado,Pontuação R2 (treinamento),Explained Variance Score (treinamento),Mean Squared Error (treinamento),Pontuação R2 (teste),Explained Variance Score (teste),Mean Squared Error (teste),Conjunto de Treino,Conjunto de Teste
0,DecisionTreeRegressor,1.000000,1.000000,5.681713e-36,0.522690,0.522699,0.017325,"(32142, 16)","(13776, 16)"
1,SVR,0.697948,0.715020,1.115072e-02,0.696669,0.712332,0.011010,"(32142, 16)","(13776, 16)"
2,KNeighborsRegressor,0.884238,0.884257,4.273524e-03,0.653379,0.653457,0.012581,"(32142, 16)","(13776, 16)"
3,LinearRegression,0.758471,0.758471,8.916399e-03,0.758395,0.758434,0.008769,"(32142, 16)","(13776, 16)"
4,XGBRegressor,0.824397,0.824397,6.482667e-03,0.770987,0.771015,0.008312,"(32142, 16)","(13776, 16)"


# Modelo escolhido

# XGBOOST

Utilizando o XGboost para uma primeira rodada de previsão

In [31]:
# Neste passo irei realizar os modelos de regressão e mostrar os resultados

from xgboost import XGBRegressor as xgb

Regression = [xgb(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7)]

resultados = { 
        'Modelo utilizado': [],
        'Conjunto de Treino': [], 
        'Conjunto de Teste': [],
        'Pontuação R2 (treinamento)': [], 
        'Pontuação R2 (teste)': [],
        'Explained Variance Score (treinamento)':[],
        'Explained Variance Score (teste)':[],
        'Mean Squared Error (treinamento)':[],
        'Mean Squared Error (teste)':[]        
            }

################################DADOS PASSADOS NO MODELO###########################################################
# ##Treino
# #Coluna targe do dataset
# target_final_data_train = train_final_data['cycle_num']
# #Colunas com os dados
# train_final_data = train_final_data.drop(['cycle_num'], axis = 1)



##################################################################################################################

Dados_normalizados = [train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)]

for clf in Regression:
    for split in Dados_normalizados:
        
        print ("Próximo modelo...\n")
        X_train, X_test, y_train, y_test = split
        
        time_train = train_classifier(clf, X_train, y_train)
        
        time_predict_train = predict_labels_train(clf, X_train, y_train)
        
        time_predict_test = predict_labels_test(clf, X_test, y_test)
        
        resultados['Modelo utilizado'].append(clf.__class__.__name__)
        resultados['Conjunto de Treino'].append(X_train.shape)
        resultados['Conjunto de Teste'].append(X_test.shape)
        resultados['Pontuação R2 (treinamento)'].append(time_predict_train["R2 score"])
        resultados['Pontuação R2 (teste)'].append(time_predict_test["R2 score"])
        resultados['Explained Variance Score (treinamento)'].append(time_predict_train["Explained Variance Score"])
        resultados['Explained Variance Score (teste)'].append(time_predict_test["Explained Variance Score"])
        resultados['Mean Squared Error (treinamento)'].append(time_predict_train["Mean Squared Error"])
        resultados['Mean Squared Error (teste)'].append(time_predict_test["Mean Squared Error"])
        

                  
Results_xgboost = pd.DataFrame(resultados)
Results_xgboost = Results_xgboost[['Modelo utilizado','Pontuação R2 (treinamento)','Explained Variance Score (treinamento)','Mean Squared Error (treinamento)',
                  'Pontuação R2 (teste)','Explained Variance Score (teste)','Mean Squared Error (teste)','Conjunto de Treino','Conjunto de Teste']]
Results_xgboost.head()


Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo XGBRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo XGBRegressor foram feitas!



,Modelo utilizado,Pontuação R2 (treinamento),Explained Variance Score (treinamento),Mean Squared Error (treinamento),Pontuação R2 (teste),Explained Variance Score (teste),Mean Squared Error (teste),Conjunto de Treino,Conjunto de Teste
0,XGBRegressor,0.824397,0.824397,0.006483,0.770987,0.771015,0.008312,"(32142, 16)","(13776, 16)"


# Ajustando o modelo do Xgboost

In [32]:
def performance_metric(y_true, y_predict):
    """ Calcular e retornar a pontuação de desempenho entre 
        valores reais e estimados baseado na métrica escolhida. """
    # TODO: Calcular a pontuação de desempenho entre 'y_true' e 'y_predict'
    score = r2_score(y_true, y_predict)
#     explainedvariancescore =  explained_variance_score(y_true, y_predict)
#     meansquarederror = mean_squared_error(y_true, y_predict)
          
    
    # Devolver a pontuação
    return score

In [33]:
def fit_model(X, y):

    from xgboost import XGBRegressor as xgb
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.2, random_state = 1)
#     cv_sets = train_test_split(X, y, train_size=0.7 , random_state = 1)
    estimator = xgb()
   
    # TODO: Gerar um dicionário para o parâmetro 'max_depth' com um alcance de 1 a 10
    Params = {'n_estimators':[100,150,200], 
          'learning_rate':[0.05,0.07,0.4],
          'colsample_bytree':[0.5,0.7,1],
          'max_depth':[5,7,10]
         }

    # TODO: Transformar 'performance_metric' em uma função de pontuação utilizando 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)
    

    # TODO: Gerar o objeto de busca em matriz
    grid = GridSearchCV( estimator=estimator, param_grid=Params, scoring=scoring_fnc, cv=cv_sets)
    #grid = GridSearchCV( estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets2)

    # Ajustar o objeto de busca em matriz com os dados para calcular o modelo ótimo
    grid = grid.fit(X, y)
   

    # Devolver o modelo ótimo depois de realizar o ajuste dos dados
    return grid.best_estimator_

print ('Tudo ok até então!')

Tudo ok até então!


In [ ]:
reg = fit_model(train_final_data, target_final_data_train_scaled)

In [ ]:
# Produzir valores para 'max_depth'
print "O parâmetro 'n_estimators' é {} para o modelo ótimo.".format(reg.get_params()['n_estimators'])
print "O parâmetro 'learning_rate' é {} para o modelo ótimo.".format(reg.get_params()['learning_rate'])
print "O parâmetro 'colsample_bytree' é {} para o modelo ótimo.".format(reg.get_params()['colsample_bytree'])
print "O parâmetro 'max_depth' é {} para o modelo ótimo.".format(reg.get_params()['max_depth'])

# Modelo Final XGBOOST

In [63]:
#Dados Finais

X_train, X_test, y_train, y_test = train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)

In [64]:
from xgboost import XGBRegressor as xgb

Regression = [xgb(n_estimators=200, learning_rate=0.05, gamma=0, subsample=0.75,colsample_bytree=1, 
                  max_depth=7)]

resultados = { 
        'Modelo utilizado': [],
        'Conjunto de Treino': [], 
        'Conjunto de Teste': [],
        'Pontuação R2 (treinamento)': [], 
        'Pontuação R2 (teste)': [],
        'Explained Variance Score (treinamento)':[],
        'Explained Variance Score (teste)':[],
        'Mean Squared Error (treinamento)':[],
        'Mean Squared Error (teste)':[]
            }

Dados_normalizados = [train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)]

for clf in Regression:
    for split in Dados_normalizados:
        
        print ("Próximo modelo...\n")
        X_train, X_test, y_train, y_test = split
        
        time_train = train_classifier(clf, X_train, y_train)
        
        time_predict_train = predict_labels_train(clf, X_train, y_train)
        
        time_predict_test = predict_labels_test(clf, X_test, y_test)
        
        resultados['Modelo utilizado'].append(clf.__class__.__name__)
        resultados['Conjunto de Treino'].append(X_train.shape)
        resultados['Conjunto de Teste'].append(X_test.shape)
        resultados['Pontuação R2 (treinamento)'].append(time_predict_train["R2 score"])
        resultados['Pontuação R2 (teste)'].append(time_predict_test["R2 score"])
        resultados['Explained Variance Score (treinamento)'].append(time_predict_train["Explained Variance Score"])
        resultados['Explained Variance Score (teste)'].append(time_predict_test["Explained Variance Score"])
        resultados['Mean Squared Error (treinamento)'].append(time_predict_train["Mean Squared Error"])
        resultados['Mean Squared Error (teste)'].append(time_predict_test["Mean Squared Error"])

                  
Results_xgboost = pd.DataFrame(resultados)
Results_xgboost = Results_xgboost[['Modelo utilizado','Pontuação R2 (treinamento)','Explained Variance Score (treinamento)','Mean Squared Error (treinamento)',
                  'Pontuação R2 (teste)','Explained Variance Score (teste)','Mean Squared Error (teste)','Conjunto de Treino','Conjunto de Teste']]
Results_xgboost.head()

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo XGBRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo XGBRegressor foram feitas!



,Modelo utilizado,Pontuação R2 (treinamento),Explained Variance Score (treinamento),Mean Squared Error (treinamento),Pontuação R2 (teste),Explained Variance Score (teste),Mean Squared Error (teste),Conjunto de Treino,Conjunto de Teste
0,XGBRegressor,0.886144,0.886144,0.004203,0.837313,0.837331,0.005905,"(32142, 19)","(13776, 19)"


In [65]:
# Params = (n_estimators = 200, 
#          learning_rate = 0.4,
#          colsample_bytree = 0.7,
#          max_depth = 5)
from xgboost import XGBRegressor as xgb        

model = xgb(n_estimators = 200,learning_rate = 0.4,colsample_bytree = 0.7,max_depth = 5)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

X_test_df = pd.DataFrame(X_test)
y_pred_final = pd.DataFrame(y_pred * target_final_data_train.max() , columns = ['y_pred'], index = X_test_df.index)

#Saving the model
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [66]:
y_pred_final.shape

(13776, 1)

In [67]:
X_test_df.shape

(13776, 19)

In [68]:
X_test_df.head()

,unit,overall setting,sensor2,sensor3,sensor4,sensor6,sensor7,sensor8,sensor9,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor20,"sensor9,14_break"
38929,185,1,0.510836,0.619269,0.285486,1.0,0.543662,0.276190,0.173858,0.0,0.400000,0.492593,0.279762,0.237667,0.465002,0.0,0.500000,0.223684,0
7359,35,1,0.359133,0.317896,0.473086,0.0,0.766197,0.247619,0.209830,0.0,0.446667,0.392593,0.258929,0.184961,0.525539,0.0,0.300000,0.407895,0
1607,8,3,0.259146,0.452178,0.209569,0.0,0.535627,0.339286,0.221120,0.0,0.355263,0.614493,0.327586,0.243961,0.346185,0.0,0.400000,0.701031,1
32915,156,4,0.428105,0.458142,0.596941,0.0,0.532164,0.214984,0.136119,0.0,0.157534,0.583333,0.210366,0.196389,0.206791,0.0,0.333333,0.500000,0
34640,165,2,0.578778,0.680204,0.679761,1.0,0.339789,0.636364,0.284511,0.0,0.875862,0.373068,0.733333,0.224325,0.763688,0.0,0.600000,0.301724,1


In [69]:
y_pred_final.head()

,y_pred
38929,93.749550
7359,91.311562
1607,177.480881
32915,67.271011
34640,160.227692


In [3]:
data_colunas = [y_pred_final,target_final_data_train,X_test_df]
# target_final_data_train

engine_train = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_train = engine_train.dropna(axis = 0, how = 'any')

engine_train.tail()

NameError: name 'y_pred_final' is not defined

In [2]:
d = []

for index, row in engine_train.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_train['d'] = d

engine_train['d'].head()

NameError: name 'engine_train' is not defined

In [1]:
def metrica(d):
    
    aux = 0
    for i in d:
        
        if (i < 0):
            aux = aux + (np.exp((-(i/13)))-1)           
        else:
            aux = aux + (np.exp(((i/10)))-1)
#         print aux
    return aux  

# Resultado Final

In [75]:
result = metrica(engine_train['d'])

print ('O resultado final para o set de treino foi de {} pontos.'.format(result))

O resultado final para o set de treino foi de 598392.726332 pontos.


# Aplicando o filtro Kalman

In [ ]:
#https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/Appendix-E-Ensemble-Kalman-Filters.ipynb

O Filtro Kalman é utilizado para filtrar os sinais dos sensores e reduzir os ruídos dos sensores. Espera-se uma melhora nos resultados.

In [ ]:
x = train_final_data['sensor2']

# Teste com o set de testes

In [1]:
X_train, X_test, y_train, y_test = train_test_split(test_final_data, target_final_data_test_scaled, 
                                                    train_size=0 , random_state = 1)

NameError: name 'train_test_split' is not defined

In [ ]:
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# y_pred = loaded_model.predict(X_test)

from xgboost import XGBRegressor as xgb        

model = xgb(n_estimators = 200,learning_rate = 0.4,colsample_bytree = 0.7,max_depth = 5)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

X_test_df = pd.DataFrame(X_test)
y_pred_final = pd.DataFrame(y_pred * target_final_data_test.max(), columns = ['y_pred'], index = X_test.index)

print(result)

In [ ]:
data_colunas = [y_pred_final,X_test_df]
# target_final_data_test

engine_test = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_test = engine_test.dropna(axis = 0, how = 'any')

engine_test.head()

In [ ]:
d = []

for index, row in engine_test.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_test['d'] = d

engine_test['d'].head()

In [ ]:
result = metrica(engine_test['d'])

print ('O resultado final para o set de treino foi de {} pontos.'.format(result))

In [ ]:
y_pred_final.head()

# Análise final com o dataset 'final_test'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(finaltest_final_data, target_final_data_finaltest_scaled, 
                                                    train_size=0 , random_state = 1)

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)

y_pred = loaded_model.predict(X_test)

X_test_df = pd.DataFrame(X_test)
y_pred_final = pd.DataFrame(y_pred * target_final_data_finaltest.max(), columns = ['y_pred'], index = X_test.index)

print(result)

In [ ]:
data_colunas = [y_pred_final,X_test_df]
# target_final_data_test

engine_finaltest = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_finaltest = engine_finaltest.dropna(axis = 0, how = 'any')

engine_finaltest.head()

In [ ]:
d = []

for index, row in engine_finaltest.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_finaltest['d'] = d

engine_finaltest['d'].head()

In [ ]:
result = metrica(engine_finaltest['d'])

print ('O resultado final para o set de treino foi de {} pontos.'.format(result))

In [ ]:
X_test_df = pd.DataFrame(X_test)
y_pred_final = pd.DataFrame(y_pred * target_final_data_finaltest.max(), columns = ['y_pred'], index = X_test.index)
d = []

data_colunas = [y_pred_final,X_test_df]
# target_final_data_test

engine_finaltest = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_finaltest = engine_finaltest.dropna(axis = 0, how = 'any')

for index, row in engine_finaltest.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_finaltest['d'] = d

engine_finaltest['d'].head()